In [1]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [3]:
!ls movielens_latest

links.csv   ratings.csv  rec_df_small.csv  small_set_movies.csv
movies.csv  README.txt	 rec_vec_df.csv    tags.csv


In [4]:
dataset_dir = 'movielens_latest' 
# https://grouplens.org/datasets/movielens/latest/

In [6]:
vocab_df = pd.read_csv(os.path.join(dataset_dir, 'small_set_movies.csv'))
vocab_df.head()

,movieId,title,genres,median_rate,imdbId,rating,timestamp
0,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,113497,4.0,845553522
1,3.0,Grumpier Old Men (1995),Comedy|Romance,3.5,113228,4.0,964981247
2,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0,114885,3.0,845554349
3,5.0,Father of the Bride Part II (1995),Comedy,3.0,113041,5.0,845553938
4,6.0,Heat (1995),Action|Crime|Thriller,3.0,113277,4.0,964982224


In [15]:
from nltk.probability import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_w = stopwords.words('english')

In [22]:
f_dist = FreqDist()
punkts = ' .,1234567890-=_`~!#$%^&*(){}[]/"'
for movie in tqdm(vocab_df['title']):
    for word in word_tokenize(movie):
        word_processed = word.lower()
        for char in punkts:
            word_processed = word_processed.replace(char, '')
        if len(word_processed)>2 and not word_processed.isdigit() and word_processed not in stop_w:
            f_dist[word_processed]+=1
f_dist

FreqDist({'man': 98, 'aka': 69, 'love': 66, 'night': 47, 'movie': 44, 'life': 44, 'story': 43, 'dead': 41, 'last': 39, 'big': 38, ...})

In [25]:
with open('knowledge_base/titles.txt', 'a') as file:
    for word in f_dist.keys():
        file.write(word+'\n')

In [3]:
os.listdir(dataset_dir)

['tags.csv',
 'rec_df_small.csv',
 'movies.csv',
 'README.txt',
 'links.csv',
 'ratings.csv',
 'rec_vec_df.csv',
 'small_set_movies.csv']

In [9]:
movies = pd.read_csv(os.path.join(dataset_dir, 'movies.csv'))
tags = pd.read_csv(os.path.join(dataset_dir, 'tags.csv'))
links = pd.read_csv(os.path.join(dataset_dir, 'links.csv'))
ratings = pd.read_csv(os.path.join(dataset_dir, 'ratings.csv'))
#movies.loc[:, 'movieId'] = movies.index

In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
data = pd.merge(ratings, movies, on='movieId')
data.to_csv('rec_df_small.csv')

In [7]:
group_by_title = data.groupby('title')

In [8]:
# data.groupby('title')['rating'].mean().sort_values(ascending=False)
# data.groupby('title')['rating'].count().sort_values(ascending=False).head() 
frame = pd.DataFrame(data.groupby('title')['rating'].mean())  
frame['num of ratings'] = pd.DataFrame(data.groupby('title')['rating'].count())
frame

,rating,num of ratings
title,,
'71 (2014),4.000000,1
'Hellboy': The Seeds of Creation (2004),4.000000,1
'Round Midnight (1986),3.500000,2
'Salem's Lot (2004),5.000000,1
'Til There Was You (1997),4.000000,2
...,...,...
eXistenZ (1999),3.863636,22
xXx (2002),2.770833,24
xXx: State of the Union (2005),2.000000,5


In [10]:
data

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [9]:
rec_df = data.pivot_table(index ='userId', columns ='title', values ='rating')
rec_df.head(2)
chosen_movies = ['Star Trek Beyond (2016)', 'Guardians of the Galaxy 2 (2017)']
for movie in chosen_movies:
    recommendation = rec_df.corrwith(rec_df[movie])
    #print(type(recommendation))
    recommendation = recommendation.dropna().sort_values(ascending = False).index[:3]
    print(recommendation)

/home/val/.local/lib/python3.7/site-packages/numpy/lib/function_base.py:2522: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/val/.local/lib/python3.7/site-packages/numpy/lib/function_base.py:2451: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


Index(['Secret of NIMH, The (1982)',
       'Girl Who Leapt Through Time, The (Toki o kakeru shôjo) (2006)',
       'Grey, The (2012)'],
      dtype='object', name='title')
Index(['Bean (1997)', 'Small Soldiers (1998)', 'Solaris (2002)'], dtype='object', name='title')


In [96]:
links.drop('tmdbId', axis=1, inplace=True)
tags.drop('timestamp', axis=1, inplace=True)

In [97]:
frame = pd.merge(movies, ratings, on='movieId').drop('timestamp', axis=1)
frame = pd.merge(frame, links, on='movieId')

In [98]:
frame

,movieId,title,genres,userId,rating,imdbId
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,114709
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,114709
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,114709
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,114709
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,114709
...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,5476944
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,5914996
100833,193585,Flint (2017),Drama,184,3.5,6397426
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,8391976


In [99]:
# del tags, ratings

In [100]:
movie_groups = frame.groupby('movieId')

In [101]:
# frame.groupby('userId').get_group(20)
# first we will set up an algorithmic approach to recommendations
# if time is left we can improve upon it with ML
# for now though we will believe the great median rating

In [102]:
movie_groups.get_group(10)['rating'].median()

3.5

In [103]:
movie_groups.get_group(10)['rating'].mean()

3.496212121212121

In [104]:
#movies.set_index('movieId', inplace=True)
for ind in tqdm(frame.loc[:, 'movieId'].unique()):
    #movieId = frame.loc[:, 'movieId'].unique()
    #print(movieId)
    rate = movie_groups.get_group(ind)['rating'].median()
    movies.loc[ind, 'median_rate'] = rate
movies

,movieId,title,genres,median_rate
0,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
1,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0
2,3.0,Grumpier Old Men (1995),Comedy|Romance,3.5
3,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0
4,5.0,Father of the Bride Part II (1995),Comedy,3.0
...,...,...,...,...
193581,NaN,NaN,NaN,4.0
193583,NaN,NaN,NaN,3.5
193585,NaN,NaN,NaN,3.5
193587,NaN,NaN,NaN,3.5


In [105]:
movies = movies.dropna()

In [106]:
movies = pd.merge(movies, links, on='movieId')
movies

,movieId,title,genres,median_rate,imdbId
0,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,113497
1,3.0,Grumpier Old Men (1995),Comedy|Romance,3.5,113228
2,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0,114885
3,5.0,Father of the Bride Part II (1995),Comedy,3.0,113041
4,6.0,Heat (1995),Action|Crime|Thriller,3.0,113277
...,...,...,...,...,...
5385,140016.0,Always Watching: A Marble Hornets Story (2015),Horror,2.0,2737926
5386,140038.0,Madly in Love (1981),Comedy,2.0,82562
5387,140162.0,Love (2015),Drama|Romance,2.5,3774694
5388,140237.0,The Runner (2015),Drama,4.5,3687398


In [107]:
movies = pd.merge(movies, ratings, on='movieId')
movies

,movieId,title,genres,median_rate,imdbId,userId,rating,timestamp
0,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,113497,6,4.0,845553522
1,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,113497,8,4.0,839463806
2,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,113497,18,3.0,1455617462
3,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,113497,19,3.0,965704331
4,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,113497,20,3.0,1054038313
...,...,...,...,...,...,...,...,...
68010,140016.0,Always Watching: A Marble Hornets Story (2015),Horror,2.0,2737926,306,2.0,1518327167
68011,140038.0,Madly in Love (1981),Comedy,2.0,82562,105,4.5,1526207048
68012,140162.0,Love (2015),Drama|Romance,2.5,3774694,318,4.5,1450717949
68013,140237.0,The Runner (2015),Drama,4.5,3687398,448,2.0,1472294240


In [108]:
def pad_to_len(value, length=7):
    value = str(value)
    while len(value)<length:
        value = '0'+value
    return value

movies.loc[:, 'imdbId'] = movies['imdbId'].apply(pad_to_len)

In [109]:
movies = movies.drop('userId', axis=1).drop_duplicates('title')
movies

,movieId,title,genres,median_rate,imdbId,rating,timestamp
0,2.0,Jumanji (1995),Adventure|Children|Fantasy,4.0,0113497,4.0,845553522
110,3.0,Grumpier Old Men (1995),Comedy|Romance,3.5,0113228,4.0,964981247
162,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0,0114885,3.0,845554349
169,5.0,Father of the Bride Part II (1995),Comedy,3.0,0113041,5.0,845553938
218,6.0,Heat (1995),Action|Crime|Thriller,3.0,0113277,4.0,964982224
...,...,...,...,...,...,...,...
68010,140016.0,Always Watching: A Marble Hornets Story (2015),Horror,2.0,2737926,2.0,1518327167
68011,140038.0,Madly in Love (1981),Comedy,2.0,0082562,4.5,1526207048
68012,140162.0,Love (2015),Drama|Romance,2.5,3774694,4.5,1450717949
68013,140237.0,The Runner (2015),Drama,4.5,3687398,2.0,1472294240


In [110]:
movies.loc[:, 'imdbId'] = movies['imdbId'].astype(str)

In [111]:
movies.to_csv('small_set_movies.csv', index=False)
#movies.to_csv('large_set_movies.csv')

In [81]:
# movies.to_json('small_set_movies.json')
movies.to_json('large_set_movies.json')

In [82]:
import gc
gc.collect()

103

In [83]:
#############################
# If you stored data before #
#############################
users_df = pd.read_csv(os.path.join(dataset_dir, 'small_set_movies.csv'))
users = users_df.groupby("userId")
print('N users:', users_df['userId'].nunique())
print('N movies:', users_df['imdbId'].nunique())
rec_df = pd.DataFrame()
for g in tqdm(users.groups):
    #print(users.get_group(g))
    #print(g)
    user_vec = users_df[users_df['userId']==g][['userId', 'movieId', 'rating', 'imdbId']]
    rec_df = rec_df.append(user_vec, ignore_index=True)
    #print(user_vec.T)
    #uservec = pd.DataFrame({'userId': g.keys()[0], 'movieId': g.values()})
    #break

N users: 610
N movies: 5380


In [84]:
rec_df

,userId,movieId,rating,imdbId
0,1,3,4.0,113228
1,1,6,4.0,113277
2,1,47,5.0,114369
3,1,50,5.0,114814
4,1,70,3.0,116367
...,...,...,...,...
68010,610,136602,4.0,2428170
68011,610,138036,3.5,1638355
68012,610,138210,3.5,4172430
68013,610,138610,1.5,2309260


In [28]:
rec_df.to_csv('rec_vec_df.csv')

In [11]:
genres = movies.genres.apply(lambda x: x.split('|') if '|' in x else [x])

In [14]:
genres_set = set()
for g in genres:
    for each in g:
        genres_set.add(each)

In [16]:
genres_set.remove('(no genres listed)')

In [17]:
genres_set

{'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}